In [1]:
import pandas as pd
import numpy as np


responses = pd.read_csv("./form-responses-2024-7-4.csv")
responses.sample()

,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
19180,4/29/2021 7:59:49,35-44,Media & Digital,Production Coordinator,Film/TV Production,"120,120",NaN,USD,NaN,Freelance,United States,Illinois,Chicago,11 - 20 years,11 - 20 years,"Professional degree (MD, JD, etc.)",Woman,White


## Data cleaning and transformation

In [2]:
new_columns = {
    "How old are you?": "age",
    "What industry do you work in?": "industry",
    "Job title": "job_title",
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "anual_salary",
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "bonuses",
    "Please indicate the currency": "currency",
    "What country do you work in?": "country",
    "If you're in the U.S., what state do you work in?": "us_state",
    "What city do you work in?": "city",
    "How many years of professional work experience do you have in your field?": "field_experience",
    "What is your highest level of education completed?": "education",
}
responses.rename(columns=new_columns, inplace=True)

In [3]:

responses['bonuses'] = responses['bonuses'].fillna(0)
responses['anual_salary'] = responses['anual_salary'].str.replace(',', '').astype(int)
responses['bonuses'] = responses['bonuses'].astype(int)

responses['salary'] = responses['anual_salary'] + responses['bonuses']
responses = responses[['age', 'industry', 'job_title', 'salary', 'currency', 'country', 'us_state', 'city', 'field_experience', 'education']]

In [4]:
responses.dropna(subset=['industry', 'job_title'], inplace=True)

def capitalize(title):
    return title.lower().strip()

responses['industry'] = responses['industry'].apply(capitalize)
responses['job_title'] = responses['job_title'].apply(capitalize)

There are 1003 different values in the industry column. I tried using the Levenshtein Distance algorithm, but it removed valuable information, so I'm going to standardize the values manually and retain only the data that I found relevant

In [5]:
keywords = [
    "Accounting, banking & finance",
    "Agriculture or forestry",
    "Art & design",
    "Business or consulting",
    "Computing or tech",
    "Education (primary/secondary)",
    "Education (higher education)",
    "Engineering or manufacturing",
    "Entertainment",
    "Government and public administration",
    "Health care",
    "Hospitality & events",
    "Insurance",
    "Law",
    "Law enforcement & security",
    "Leisure, sport & tourism",
    "Marketing, advertising & pr",
    "Media & digital",
    "Nonprofits",
    "Property or construction",
    "Recruitment or hr",
    "Retail",
    "Sales",
    "Social work",
    "Transport or logistics",
    "Utilities & telecommunications",

]

keywords_containing = [
    "academia",
    "aerospace",
    "research",
    "architect",
    "auto",
    "biotech",
    "construction",
    "consulting",
    "finance",
    "telecommunications",
    "energy",
    "manufacturing",
    "environment",
    "food",
    "government",
    "health",
    "librari",
    "library",
    "mining",
    "museum",
    "nonprofit",
    "oil",
    "software",
    "pharma",
    "politic",
    "public",
    "publishing",
    "real estate",
    "science",
    "scienti",
    "vet",
]


keyword_set = set(keywords)
keyword_containing_set = set(keywords_containing)


def replace_with_keywords(value, keyword_set, keyword_containing_set):

    for keyword in keyword_set:
        if value.lower().startswith(keyword.lower()):
            return keyword.capitalize()

    for keyword in keyword_containing_set:
        if keyword.lower() in value.lower():
            return keyword.capitalize()

    return np.NaN


responses["industry"] = responses["industry"].apply(
    lambda x: replace_with_keywords(x, keyword_set, keyword_containing_set)
)

responses = responses[responses["industry"].notna()]